### Prework: Load Cleaned Up Data from Deliverable 2

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import balanced_accuracy_score
from imblearn.ensemble import EasyEnsembleClassifier
import pandas as pd
import tensorflow as tf

#  Import cleaned data from prior activity.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [3]:
# Drop the non-beneficial ID columns, 'EIN', 'USE_CASE', 'SPECIAL_CONSIDERATIONS','affiliation'.
#  YOUR CODE GOES HERE
drop_cols = ['EIN','STATUS', 'AFFILIATION', 'USE_CASE', 'SPECIAL_CONSIDERATIONS']
application_df = application_df.drop(drop_cols, axis=1)
application_df.head()

,NAME,APPLICATION_TYPE,CLASSIFICATION,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,C1000,Association,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,C2000,Co-operative,1-9999,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,C3000,Association,0,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,C2000,Trust,10000-24999,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,C1000,Trust,100000-499999,142590,1


In [4]:
# Determine which values to replace if counts are less than ...?
#  YOUR CODE GOES HERE
app_type_counts = application_df.APPLICATION_TYPE.value_counts()
replace_application = list(app_type_counts[app_type_counts < 300].index)
# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
classification_counts = application_df.CLASSIFICATION.value_counts()
replace_class = list(classification_counts[classification_counts < 1000].index)
# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [6]:
name_counts = application_df.NAME.value_counts()
replace_class = list(name_counts[name_counts < 250].index)
# Replace in dataframe
for cls in replace_class:
    application_df.NAME = application_df.NAME.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.NAME.value_counts()

Other                                                28539
PARENT BOOSTER USA INC                                1260
TOPS CLUB INC                                          765
UNITED STATES BOWLING CONGRESS INC                     700
WASHINGTON STATE UNIVERSITY                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC        408
PTA TEXAS CONGRESS                                     368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC          331
ALPHA PHI SIGMA                                        313
TOASTMASTERS INTERNATIONAL                             293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS      287
LITTLE LEAGUE BASEBALL INC                             277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS               266
Name: NAME, dtype: int64

In [7]:
application_df.nunique()

NAME                  13
APPLICATION_TYPE       9
CLASSIFICATION         6
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [8]:
# Generate our categorical variable lists
#  YOUR CODE GOES HERE
# categorical data will be "object" datatype
application_cat = [cur_col for cur_col in application_df.columns if application_df[cur_col].dtype == 'O' ]
print(application_cat)

['NAME', 'APPLICATION_TYPE', 'CLASSIFICATION', 'ORGANIZATION', 'INCOME_AMT']


In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_INTERNATIONAL ASSOCIATION OF LIONS CLUBS,NAME_LITTLE LEAGUE BASEBALL INC,NAME_MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS,NAME_Other,NAME_PARENT BOOSTER USA INC,NAME_PTA TEXAS CONGRESS,NAME_SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC,NAME_TOASTMASTERS INTERNATIONAL,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
#  YOUR CODE GOES HERE
merged_df = pd.merge(application_df, encode_df, left_index=True, right_index=True)
print("merged shape", merged_df.shape)
merged_df = merged_df.drop(application_cat, axis=1)
print("merged ( dropped ) shape", merged_df.shape)
merged_df.head()

merged shape (34299, 48)
merged ( dropped ) shape (34299, 43)


,ASK_AMT,IS_SUCCESSFUL,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_INTERNATIONAL ASSOCIATION OF LIONS CLUBS,NAME_LITTLE LEAGUE BASEBALL INC,NAME_MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS,NAME_Other,NAME_PARENT BOOSTER USA INC,NAME_PTA TEXAS CONGRESS,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = merged_df.IS_SUCCESSFUL
X = merged_df.drop(["IS_SUCCESSFUL"], axis=1)

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, random_state=1)
model_rf = model_rf.fit(X_train_scaled, y_train)
model_rf.score(X_test_scaled, y_test)

0.7145189504373178

In [14]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = model_rf.feature_importances_
# print the importances in descending order
sorted(zip(importances, X.columns), reverse=True)

[(0.42102779200588075, 'ASK_AMT'),
 (0.13916861038693, 'NAME_Other'),
 (0.039625895075160525, 'ORGANIZATION_Association'),
 (0.032687466729236184, 'ORGANIZATION_Trust'),
 (0.02490451182350541, 'APPLICATION_TYPE_T19'),
 (0.024822017173731212, 'NAME_TOASTMASTERS INTERNATIONAL'),
 (0.019644630937210847, 'CLASSIFICATION_C2100'),
 (0.01933239857733526, 'APPLICATION_TYPE_T10'),
 (0.019295719163497895, 'CLASSIFICATION_C2000'),
 (0.018755757924900526, 'APPLICATION_TYPE_T4'),
 (0.018537648010077443, 'NAME_UNITED STATES BOWLING CONGRESS INC'),
 (0.017996365615484466, 'CLASSIFICATION_Other'),
 (0.01641383985986102, 'NAME_PARENT BOOSTER USA INC'),
 (0.014949368261195337, 'NAME_TOPS CLUB INC'),
 (0.014590377345101853, 'APPLICATION_TYPE_T3'),
 (0.013677757319817607, 'APPLICATION_TYPE_T5'),
 (0.012770592918767452, 'CLASSIFICATION_C1000'),
 (0.012510110184707896, 'CLASSIFICATION_C1200'),
 (0.011726498443946297,
  'NAME_MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS'),
 (0.011282656803601729, 'NAME_

### Let's try the last activity model

In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [16]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

### Prior work: Compile, Train and Evaluate the Model

In [17]:
## Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_count = len(X.columns)
units1_count =  int(input_count/2)
units2_count = int(units1_count/2)

nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=units1_count, activation="relu", input_dim=input_count))
nn.add(tf.keras.layers.Dense(units=units2_count, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)


Epoch 1/100
804/804 [==============================] - 1s 732us/step - loss: 0.5838 - accuracy: 0.6889
Epoch 2/100
195/804 [======>.......................] - ETA: 0s - loss: 0.5317 - accuracy: 0.7332
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 806us/step - loss: 0.5409 - accuracy: 0.7262
Epoch 3/100
335/804 [===========>..................] - ETA: 0s - loss: 0.5366 - accuracy: 0.7278
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 868us/step - loss: 0.5379 - accuracy: 0.7268
Epoch 4/100
564/804 [====================>.........] - ETA: 0s - loss: 0.5380 - accuracy: 0.7260
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 936us/step - loss: 0.5366 - accuracy: 0.7273
Epoch 5/100
767/804 [===========================>..] - ETA: 0s - loss: 0.5358 - accuracy: 0.7278
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 

232/804 [=======>......................] - ETA: 0s - loss: 0.5264 - accuracy: 0.7338
Epoch 00038: saving model to checkpoints\weights.38.hdf5
804/804 [==============================] - 1s 864us/step - loss: 0.5309 - accuracy: 0.7296
Epoch 39/100
408/804 [==============>...............] - ETA: 0s - loss: 0.5291 - accuracy: 0.7311
Epoch 00039: saving model to checkpoints\weights.39.hdf5
804/804 [==============================] - 1s 907us/step - loss: 0.5310 - accuracy: 0.7291
Epoch 40/100
575/804 [====================>.........] - ETA: 0s - loss: 0.5322 - accuracy: 0.7305
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 788us/step - loss: 0.5313 - accuracy: 0.7299
Epoch 41/100
804/804 [==============================] - 1s 761us/step - loss: 0.5309 - accuracy: 0.73010s - loss: 0.5287 - accu
Epoch 42/100
  1/804 [..............................] - ETA: 0s - loss: 0.4244 - accuracy: 0.8438
Epoch 00042: saving model to checkpoints\weights.

804/804 [==============================] - 1s 791us/step - loss: 0.5298 - accuracy: 0.7301
Epoch 75/100
473/804 [================>.............] - ETA: 0s - loss: 0.5291 - accuracy: 0.7312
Epoch 00075: saving model to checkpoints\weights.75.hdf5
804/804 [==============================] - 1s 780us/step - loss: 0.5298 - accuracy: 0.7304
Epoch 76/100
681/804 [========================>.....] - ETA: 0s - loss: 0.5301 - accuracy: 0.7308
Epoch 00076: saving model to checkpoints\weights.76.hdf5
804/804 [==============================] - 1s 756us/step - loss: 0.5299 - accuracy: 0.7307
Epoch 77/100
804/804 [==============================] - 1s 836us/step - loss: 0.5296 - accuracy: 0.7306
Epoch 78/100
 59/804 [=>............................] - ETA: 0s - loss: 0.5183 - accuracy: 0.7394
Epoch 00078: saving model to checkpoints\weights.78.hdf5
804/804 [==============================] - 1s 800us/step - loss: 0.5295 - accuracy: 0.7297
Epoch 79/100
252/804 [========>.....................] - ETA: 0s - l

In [18]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.5382367372512817, Accuracy: 0.7266472578048706


### Let's try more layers and nodes

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_count = len(X.columns)
units1_count = 256
units2_count = 128
units3_count = 64
units4_count = 32

nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=units1_count, activation="relu", input_dim=input_count))
nn.add(tf.keras.layers.Dense(units=units2_count, activation="relu"))
nn.add(tf.keras.layers.Dense(units=units3_count, activation="relu"))
nn.add(tf.keras.layers.Dense(units=units4_count, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)


Epoch 1/100
571/804 [====================>.........] - ETA: 0s - loss: 0.5506 - accuracy: 0.7186
Epoch 00001: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5491 - accuracy: 0.7198
Epoch 2/100
783/804 [============================>.] - ETA: 0s - loss: 0.5401 - accuracy: 0.7265
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5406 - accuracy: 0.7262
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5384 - accuracy: 0.7278: 0s
Epoch 4/100
174/804 [=====>........................] - ETA: 0s - loss: 0.5304 - accuracy: 0.7313
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5372 - accuracy: 0.7279
Epoch 5/100
362/804 [============>.................] - ETA: 0s - loss: 0.5415 - accuracy: 0.7230
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [===

804/804 [==============================] - 1s 1ms/step - loss: 0.5291 - accuracy: 0.7312
Epoch 37/100
637/804 [======================>.......] - ETA: 0s - loss: 0.5247 - accuracy: 0.73 - ETA: 0s - loss: 0.5243 - accuracy: 0.7346
Epoch 00037: saving model to checkpoints\weights.37.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5287 - accuracy: 0.7314
Epoch 38/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5286 - accuracy: 0.7316
Epoch 39/100
 43/804 [>.............................] - ETA: 0s - loss: 0.5198 - accuracy: 0.7406
Epoch 00039: saving model to checkpoints\weights.39.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5292 - accuracy: 0.7308
Epoch 40/100
213/804 [======>.......................] - ETA: 0s - loss: 0.5250 - accuracy: 0.7325
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5284 - accuracy: 0.7318: 0s - loss: 0.5288 - ac
Epoch 41/10

804/804 [==============================] - 1s 1ms/step - loss: 0.5262 - accuracy: 0.7321: 0s - loss: 0.5244 - accuracy: 0.73 - ETA: 0s - loss: 0.5258 - accuracy
Epoch 72/100
478/804 [================>.............] - ETA: 0s - loss: 0.5295 - accuracy: 0.72 - ETA: 0s - loss: 0.5280 - accuracy: 0.7302
Epoch 00072: saving model to checkpoints\weights.72.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5265 - accuracy: 0.7321
Epoch 73/100
679/804 [========================>.....] - ETA: 0s - loss: 0.5243 - accuracy: 0.7329 ETA: 0s - loss: 0.5240 - accura
Epoch 00073: saving model to checkpoints\weights.73.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5264 - accuracy: 0.7321
Epoch 74/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5262 - accuracy: 0.7321: 0s - loss: 0.5228 - 
Epoch 75/100
 84/804 [==>...........................] - ETA: 0s - loss: 0.5200 - accuracy: 0.7467
Epoch 00075: saving model to checkpoints\weights.75.hdf

In [20]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.5585751533508301, Accuracy: 0.7250145673751831


### Let's try beefing up the nodes

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_count = len(X.columns)
units1_count = 500
units2_count = 500
units3_count = 200
units4_count = 100

nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=units1_count, activation="relu", input_dim=input_count))
nn.add(tf.keras.layers.Dense(units=units2_count, activation="relu"))
nn.add(tf.keras.layers.Dense(units=units3_count, activation="relu"))
nn.add(tf.keras.layers.Dense(units=units4_count, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)


Epoch 1/100
181/804 [=====>........................] - ETA: 1s - loss: 0.5710 - accuracy: 0.7077
Epoch 00001: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5523 - accuracy: 0.7188
Epoch 2/100
376/804 [=============>................] - ETA: 0s - loss: 0.5441 - accuracy: 0.7274
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5422 - accuracy: 0.7260: 0s - loss: 0.5425 - accuracy
Epoch 3/100
590/804 [=====================>........] - ETA: 0s - loss: 0.5370 - accuracy: 0.7273
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5393 - accuracy: 0.7269
Epoch 4/100
777/804 [===========================>..] - ETA: 0s - loss: 0.5385 - accuracy: 0.7265
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5380 - accuracy: 0.726

249/804 [========>.....................] - ETA: 1s - loss: 0.5318 - accuracy: 0.7287
Epoch 00037: saving model to checkpoints\weights.37.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5296 - accuracy: 0.7311: 0s - loss: 0.5296 - accuracy: 0.73
Epoch 38/100
444/804 [===============>..............] - ETA: 0s - loss: 0.5259 - accuracy: 0.7365 ETA: 1s - loss: 0.5262 - ac
Epoch 00038: saving model to checkpoints\weights.38.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5300 - accuracy: 0.7313: 0s - loss: 0
Epoch 39/100
635/804 [======================>.......] - ETA: 0s - loss: 0.5316 - accuracy: 0.7295
Epoch 00039: saving model to checkpoints\weights.39.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5302 - accuracy: 0.7310
Epoch 40/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5295 - accuracy: 0.7314: 0s -
Epoch 41/100
 24/804 [..............................] - ETA: 1s - loss: 0.5297 - accuracy: 0.72

804/804 [==============================] - 2s 2ms/step - loss: 0.5269 - accuracy: 0.7314
Epoch 73/100
289/804 [=========>....................] - ETA: 1s - loss: 0.5274 - accuracy: 0.7296
Epoch 00073: saving model to checkpoints\weights.73.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5268 - accuracy: 0.7315: 0s - loss: 0.5268 - accuracy: 0.73
Epoch 74/100
491/804 [=================>............] - ETA: 0s - loss: 0.5246 - accuracy: 0.7331
Epoch 00074: saving model to checkpoints\weights.74.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5277 - accuracy: 0.7308
Epoch 75/100
681/804 [========================>.....] - ETA: 0s - loss: 0.5248 - accuracy: 0.7325
Epoch 00075: saving model to checkpoints\weights.75.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5271 - accuracy: 0.7308
Epoch 76/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5270 - accuracy: 0.7312: 0s - loss: 0.5261 - ac
Epoch 77/100
 76/

In [22]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.5722206234931946, Accuracy: 0.7258309125900269


### Let's try simplifying since doing more seems to make worse

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_count = len(X.columns)
units1_count = 12

nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=units1_count, activation="relu", input_dim=input_count))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)


Epoch 1/100
782/804 [============================>.] - ETA: 0s - loss: 0.5993 - accuracy: 0.6806
Epoch 00001: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 1s 726us/step - loss: 0.5982 - accuracy: 0.6810
Epoch 2/100
804/804 [==============================] - 1s 669us/step - loss: 0.5508 - accuracy: 0.7233
Epoch 3/100
147/804 [====>.........................] - ETA: 0s - loss: 0.5479 - accuracy: 0.7209
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 703us/step - loss: 0.5476 - accuracy: 0.7257
Epoch 4/100
380/804 [=============>................] - ETA: 0s - loss: 0.5462 - accuracy: 0.7267
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 677us/step - loss: 0.5457 - accuracy: 0.7278
Epoch 5/100
547/804 [===================>..........] - ETA: 0s - loss: 0.5459 - accuracy: 0.7259
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 

804/804 [==============================] - 1s 692us/step - loss: 0.5326 - accuracy: 0.7290
Epoch 39/100
218/804 [=======>......................] - ETA: 0s - loss: 0.5334 - accuracy: 0.7265
Epoch 00039: saving model to checkpoints\weights.39.hdf5
804/804 [==============================] - 1s 709us/step - loss: 0.5328 - accuracy: 0.7290
Epoch 40/100
442/804 [===============>..............] - ETA: 0s - loss: 0.5313 - accuracy: 0.7306
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 727us/step - loss: 0.5327 - accuracy: 0.7295
Epoch 41/100
591/804 [=====================>........] - ETA: 0s - loss: 0.5341 - accuracy: 0.7272
Epoch 00041: saving model to checkpoints\weights.41.hdf5
804/804 [==============================] - 1s 695us/step - loss: 0.5327 - accuracy: 0.7291
Epoch 42/100
804/804 [==============================] - 1s 658us/step - loss: 0.5322 - accuracy: 0.7297
Epoch 43/100
  1/804 [..............................] - ETA: 0s - l

804/804 [==============================] - 1s 691us/step - loss: 0.5316 - accuracy: 0.72970s - loss: 0.5254 - accu
Epoch 76/100
446/804 [===============>..............] - ETA: 0s - loss: 0.5331 - accuracy: 0.7279
Epoch 00076: saving model to checkpoints\weights.76.hdf5
804/804 [==============================] - 1s 711us/step - loss: 0.5318 - accuracy: 0.7296
Epoch 77/100
667/804 [=======================>......] - ETA: 0s - loss: 0.5324 - accuracy: 0.7291
Epoch 00077: saving model to checkpoints\weights.77.hdf5
804/804 [==============================] - 1s 700us/step - loss: 0.5317 - accuracy: 0.7294
Epoch 78/100
804/804 [==============================] - 1s 669us/step - loss: 0.5314 - accuracy: 0.7294
Epoch 79/100
 78/804 [=>............................] - ETA: 0s - loss: 0.5187 - accuracy: 0.7396
Epoch 00079: saving model to checkpoints\weights.79.hdf5
804/804 [==============================] - 1s 699us/step - loss: 0.5317 - accuracy: 0.7287
Epoch 80/100
228/804 [=======>.............

In [24]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.537021279335022, Accuracy: 0.727580189704895


### trying sigmoid as next layer

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_count = len(X.columns)
units1_count = 12

nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=units1_count, activation="selu", input_dim=input_count))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)


Epoch 1/100
373/804 [============>.................] - ETA: 0s - loss: 0.6539 - accuracy: 0.6647
Epoch 00001: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 1s 681us/step - loss: 0.6025 - accuracy: 0.6885
Epoch 2/100
548/804 [===================>..........] - ETA: 0s - loss: 0.5493 - accuracy: 0.7140
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 704us/step - loss: 0.5464 - accuracy: 0.7174
Epoch 3/100
766/804 [===========================>..] - ETA: 0s - loss: 0.5438 - accuracy: 0.7205
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 675us/step - loss: 0.5440 - accuracy: 0.7206
Epoch 4/100
804/804 [==============================] - 1s 658us/step - loss: 0.5422 - accuracy: 0.7229
Epoch 5/100
165/804 [=====>........................] - ETA: 0s - loss: 0.5351 - accuracy: 0.7286
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 

804/804 [==============================] - 1s 698us/step - loss: 0.5352 - accuracy: 0.7269
Epoch 38/100
629/804 [======================>.......] - ETA: 0s - loss: 0.5351 - accuracy: 0.7261 ETA: 0s - loss: 0.5317 - accuracy
Epoch 00038: saving model to checkpoints\weights.38.hdf5
804/804 [==============================] - 1s 667us/step - loss: 0.5352 - accuracy: 0.7270
Epoch 39/100
804/804 [==============================] - 1s 704us/step - loss: 0.5349 - accuracy: 0.7271
Epoch 40/100
  1/804 [..............................] - ETA: 0s - loss: 0.5480 - accuracy: 0.6875
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 675us/step - loss: 0.5348 - accuracy: 0.7277
Epoch 41/100
233/804 [=======>......................] - ETA: 0s - loss: 0.5313 - accuracy: 0.7251
Epoch 00041: saving model to checkpoints\weights.41.hdf5
804/804 [==============================] - 1s 650us/step - loss: 0.5344 - accuracy: 0.7264
Epoch 42/100
375/804 [===========

672/804 [========================>.....] - ETA: 0s - loss: 0.5340 - accuracy: 0.7275
Epoch 00074: saving model to checkpoints\weights.74.hdf5
804/804 [==============================] - 1s 696us/step - loss: 0.5337 - accuracy: 0.7277
Epoch 75/100
804/804 [==============================] - 1s 656us/step - loss: 0.5334 - accuracy: 0.7268
Epoch 76/100
 73/804 [=>............................] - ETA: 0s - loss: 0.5219 - accuracy: 0.7286
Epoch 00076: saving model to checkpoints\weights.76.hdf5
804/804 [==============================] - 1s 762us/step - loss: 0.5336 - accuracy: 0.7278
Epoch 77/100
282/804 [=========>....................] - ETA: 0s - loss: 0.5345 - accuracy: 0.7247
Epoch 00077: saving model to checkpoints\weights.77.hdf5
804/804 [==============================] - 1s 804us/step - loss: 0.5335 - accuracy: 0.7278
Epoch 78/100
485/804 [=================>............] - ETA: 0s - loss: 0.5380 - accuracy: 0.7256
Epoch 00078: saving model to checkpoints\weights.78.hdf5
804/804 [=======

In [26]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.5368872880935669, Accuracy: 0.7273469567298889


### trying softmax

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_count = len(X.columns)
units1_count = 12

nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=units1_count, activation="softmax", input_dim=input_count))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)


Epoch 1/100
804/804 [==============================] - 1s 694us/step - loss: 0.6335 - accuracy: 0.6822
Epoch 2/100
191/804 [======>.......................] - ETA: 0s - loss: 0.5865 - accuracy: 0.7156
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 742us/step - loss: 0.5756 - accuracy: 0.7172
Epoch 3/100
378/804 [=============>................] - ETA: 0s - loss: 0.5600 - accuracy: 0.7207
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 664us/step - loss: 0.5582 - accuracy: 0.7236
Epoch 4/100
536/804 [===================>..........] - ETA: 0s - loss: 0.5517 - accuracy: 0.7277
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 671us/step - loss: 0.5512 - accuracy: 0.7268
Epoch 5/100
782/804 [============================>.] - ETA: 0s - loss: 0.5470 - accuracy: 0.7268
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 

804/804 [==============================] - 1s 638us/step - loss: 0.5327 - accuracy: 0.7303
Epoch 39/100
398/804 [=============>................] - ETA: 0s - loss: 0.5324 - accuracy: 0.7288
Epoch 00039: saving model to checkpoints\weights.39.hdf5
804/804 [==============================] - 1s 649us/step - loss: 0.5326 - accuracy: 0.7297
Epoch 40/100
576/804 [====================>.........] - ETA: 0s - loss: 0.5308 - accuracy: 0.7303
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 635us/step - loss: 0.5326 - accuracy: 0.7304
Epoch 41/100
804/804 [==============================] - 0s 619us/step - loss: 0.5325 - accuracy: 0.73030s - loss: 0.5308 - accu
Epoch 42/100
  1/804 [..............................] - ETA: 0s - loss: 0.6410 - accuracy: 0.6250
Epoch 00042: saving model to checkpoints\weights.42.hdf5
804/804 [==============================] - 1s 667us/step - loss: 0.5325 - accuracy: 0.7298
Epoch 43/100
162/804 [=====>...............

456/804 [================>.............] - ETA: 0s - loss: 0.5305 - accuracy: 0.7289
Epoch 00075: saving model to checkpoints\weights.75.hdf5
804/804 [==============================] - 1s 677us/step - loss: 0.5310 - accuracy: 0.7307
Epoch 76/100
633/804 [======================>.......] - ETA: 0s - loss: 0.5302 - accuracy: 0.7304 ETA: 0s - loss: 0.5179 - accu
Epoch 00076: saving model to checkpoints\weights.76.hdf5
804/804 [==============================] - 1s 660us/step - loss: 0.5311 - accuracy: 0.7306
Epoch 77/100
804/804 [==============================] - 1s 650us/step - loss: 0.5308 - accuracy: 0.7308
Epoch 78/100
 77/804 [=>............................] - ETA: 0s - loss: 0.5336 - accuracy: 0.7378
Epoch 00078: saving model to checkpoints\weights.78.hdf5
804/804 [==============================] - 1s 686us/step - loss: 0.5310 - accuracy: 0.7306
Epoch 79/100
243/804 [========>.....................] - ETA: 0s - loss: 0.5306 - accuracy: 0.7280
Epoch 00079: saving model to checkpoints\we

In [28]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.5354726314544678, Accuracy: 0.7269970774650574


### relu followed by softmax

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_count = len(X.columns)
units1_count = 500
units2_count = 200


nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=units1_count, activation="relu", input_dim=input_count))
nn.add(tf.keras.layers.Dense(units=units2_count, activation="softmax"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)


Epoch 1/100
568/804 [====================>.........] - ETA: 0s - loss: 0.6227 - accuracy: 0.7142
Epoch 00001: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6137 - accuracy: 0.7172
Epoch 2/100
765/804 [===========================>..] - ETA: 0s - loss: 0.5773 - accuracy: 0.7254
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5762 - accuracy: 0.7259
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5637 - accuracy: 0.7265
Epoch 4/100
171/804 [=====>........................] - ETA: 0s - loss: 0.5624 - accuracy: 0.7246
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5558 - accuracy: 0.7265
Epoch 5/100
371/804 [============>.................] - ETA: 0s - loss: 0.5518 - accuracy: 0.7246
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [=======

 43/804 [>.............................] - ETA: 0s - loss: 0.5401 - accuracy: 0.7246
Epoch 00039: saving model to checkpoints\weights.39.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5373 - accuracy: 0.7301
Epoch 40/100
228/804 [=======>......................] - ETA: 0s - loss: 0.5319 - accuracy: 0.7327
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7292
Epoch 41/100
418/804 [==============>...............] - ETA: 0s - loss: 0.5367 - accuracy: 0.7282
Epoch 00041: saving model to checkpoints\weights.41.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5369 - accuracy: 0.7295
Epoch 42/100
632/804 [======================>.......] - ETA: 0s - loss: 0.5359 - accuracy: 0.7310
Epoch 00042: saving model to checkpoints\weights.42.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7300
Epoch 43/100
804/804 [===============

291/804 [=========>....................] - ETA: 0s - loss: 0.5315 - accuracy: 0.7308
Epoch 00076: saving model to checkpoints\weights.76.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5356 - accuracy: 0.7302
Epoch 77/100
462/804 [================>.............] - ETA: 0s - loss: 0.5313 - accuracy: 0.7323
Epoch 00077: saving model to checkpoints\weights.77.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5350 - accuracy: 0.7301
Epoch 78/100
668/804 [=======================>......] - ETA: 0s - loss: 0.5363 - accuracy: 0.7300
Epoch 00078: saving model to checkpoints\weights.78.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5352 - accuracy: 0.7304
Epoch 79/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5349 - accuracy: 0.7302
Epoch 80/100
 42/804 [>.............................] - ETA: 0s - loss: 0.5422 - accuracy: 0.7262
Epoch 00080: saving model to checkpoints\weights.80.hdf5
804/804 [===============

In [30]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.5488525032997131, Accuracy: 0.7240816354751587


### Trying tanh

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_count = len(X.columns)
units1_count = 500
units2_count = 200


nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=units1_count, activation="relu", input_dim=input_count))
nn.add(tf.keras.layers.Dense(units=units2_count, activation="tanh"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)


Epoch 1/100
190/804 [======>.......................] - ETA: 0s - loss: 0.5887 - accuracy: 0.6895
Epoch 00001: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5570 - accuracy: 0.7134
Epoch 2/100
378/804 [=============>................] - ETA: 0s - loss: 0.5406 - accuracy: 0.7276
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5423 - accuracy: 0.7235
Epoch 3/100
573/804 [====================>.........] - ETA: 0s - loss: 0.5375 - accuracy: 0.7275
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5405 - accuracy: 0.7238
Epoch 4/100
780/804 [============================>.] - ETA: 0s - loss: 0.5397 - accuracy: 0.7250
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5394 - accuracy: 0.7253
Epoch 5/100
804/804 [=======

804/804 [==============================] - 1s 1ms/step - loss: 0.5327 - accuracy: 0.7291
Epoch 39/100
638/804 [======================>.......] - ETA: 0s - loss: 0.5345 - accuracy: 0.7260
Epoch 00039: saving model to checkpoints\weights.39.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5325 - accuracy: 0.7288
Epoch 40/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5324 - accuracy: 0.7285
Epoch 41/100
  1/804 [..............................] - ETA: 1s - loss: 0.7503 - accuracy: 0.6250
Epoch 00041: saving model to checkpoints\weights.41.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5326 - accuracy: 0.7288: 0s - los
Epoch 42/100
207/804 [======>.......................] - ETA: 0s - loss: 0.5299 - accuracy: 0.7369
Epoch 00042: saving model to checkpoints\weights.42.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5327 - accuracy: 0.7296
Epoch 43/100
426/804 [==============>...............] - ETA: 0s - l

804/804 [==============================] - 1s 1ms/step - loss: 0.5313 - accuracy: 0.7302
Epoch 76/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5307 - accuracy: 0.7301
Epoch 77/100
 83/804 [==>...........................] - ETA: 0s - loss: 0.5358 - accuracy: 0.7357
Epoch 00077: saving model to checkpoints\weights.77.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5319 - accuracy: 0.7297
Epoch 78/100
256/804 [========>.....................] - ETA: 0s - loss: 0.5352 - accuracy: 0.7246
Epoch 00078: saving model to checkpoints\weights.78.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5319 - accuracy: 0.7286: 0s - loss: 0.5324 - accuracy: 0.
Epoch 79/100
477/804 [================>.............] - ETA: 0s - loss: 0.5309 - accuracy: 0.7328
Epoch 00079: saving model to checkpoints\weights.79.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5309 - accuracy: 0.7306: 0s - loss: 0.5291 - accu
Epoch 80/100
671/

In [32]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.5443599224090576, Accuracy: 0.7218658924102783


### More sigmoid help?  

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_count = len(X.columns)
units1_count = 500
units2_count = 200


nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=units1_count, activation="relu", input_dim=input_count))
nn.add(tf.keras.layers.Dense(units=units2_count, activation="sigmoid"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)


Epoch 1/100
764/804 [===========================>..] - ETA: 0s - loss: 0.5532 - accuracy: 0.7147
Epoch 00001: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7145
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5429 - accuracy: 0.7220
Epoch 3/100
176/804 [=====>........................] - ETA: 0s - loss: 0.5355 - accuracy: 0.7346
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5406 - accuracy: 0.7240
Epoch 4/100
388/804 [=============>................] - ETA: 0s - loss: 0.5366 - accuracy: 0.7301
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5391 - accuracy: 0.7249
Epoch 5/100
563/804 [====================>.........] - ETA: 0s - loss: 0.5369 - accuracy: 0.7257 ETA: 0s - loss: 0
Epoch 00005: saving model to checkpoints\weights.05.hdf

 43/804 [>.............................] - ETA: 0s - loss: 0.5335 - accuracy: 0.7318
Epoch 00038: saving model to checkpoints\weights.38.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5282 - accuracy: 0.7304: 0s - loss: 0.5253 - accu - ETA: 0s - loss: 0.5
Epoch 39/100
243/804 [========>.....................] - ETA: 0s - loss: 0.5275 - accuracy: 0.7306
Epoch 00039: saving model to checkpoints\weights.39.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5277 - accuracy: 0.7310
Epoch 40/100
430/804 [===============>..............] - ETA: 0s - loss: 0.5262 - accuracy: 0.7316
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5278 - accuracy: 0.7308
Epoch 41/100
636/804 [======================>.......] - ETA: 0s - loss: 0.5271 - accuracy: 0.7322
Epoch 00041: saving model to checkpoints\weights.41.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5276 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5264 - accuracy: 0.7315
Epoch 75/100
272/804 [=========>....................] - ETA: 0s - loss: 0.5238 - accuracy: 0.7325
Epoch 00075: saving model to checkpoints\weights.75.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5264 - accuracy: 0.7320
Epoch 76/100
491/804 [=================>............] - ETA: 0s - loss: 0.5269 - accuracy: 0.7325
Epoch 00076: saving model to checkpoints\weights.76.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5265 - accuracy: 0.7315
Epoch 77/100
685/804 [========================>.....] - ETA: 0s - loss: 0.5259 - accuracy: 0.7323
Epoch 00077: saving model to checkpoints\weights.77.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5260 - accuracy: 0.7320
Epoch 78/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5260 - accuracy: 0.7319: 0s - loss: 0.5216  - ETA: 0s - loss: 0.5251 - accuracy: 
Epoch 79/100
 80/80

In [34]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.5440492033958435, Accuracy: 0.7245481014251709


## more sigmoid layers?

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_count = len(X.columns)
units1_count = 500
units2_count = 200
units3_count = 100
units4_count = 50

nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=units1_count, activation="relu", input_dim=input_count))
nn.add(tf.keras.layers.Dense(units=units2_count, activation="sigmoid"))
nn.add(tf.keras.layers.Dense(units=units3_count, activation='sigmoid'))
nn.add(tf.keras.layers.Dense(units=units4_count, activation='sigmoid'))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)


Epoch 1/100
386/804 [=============>................] - ETA: 0s - loss: 0.5706 - accuracy: 0.7043
Epoch 00001: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5604 - accuracy: 0.7141
Epoch 2/100
569/804 [====================>.........] - ETA: 0s - loss: 0.5464 - accuracy: 0.7237
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5444 - accuracy: 0.7261
Epoch 3/100
782/804 [============================>.] - ETA: 0s - loss: 0.5398 - accuracy: 0.7262
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5401 - accuracy: 0.7259
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5390 - accuracy: 0.7268
Epoch 5/100
162/804 [=====>........................] - ETA: 1s - loss: 0.5338 - accuracy: 0.7321
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [=======

804/804 [==============================] - 1s 2ms/step - loss: 0.5287 - accuracy: 0.7313
Epoch 38/100
650/804 [=======================>......] - ETA: 0s - loss: 0.5285 - accuracy: 0.7322 ETA: 0s - loss: 0.5228 - ac
Epoch 00038: saving model to checkpoints\weights.38.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5279 - accuracy: 0.7316
Epoch 39/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5282 - accuracy: 0.7311
Epoch 40/100
 34/804 [>.............................] - ETA: 1s - loss: 0.5234 - accuracy: 0.7381
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5280 - accuracy: 0.7311
Epoch 41/100
235/804 [=======>......................] - ETA: 0s - loss: 0.5234 - accuracy: 0.7384
Epoch 00041: saving model to checkpoints\weights.41.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5280 - accuracy: 0.7308
Epoch 42/100
418/804 [==============>............

804/804 [==============================] - 1s 2ms/step - loss: 0.5262 - accuracy: 0.7322
Epoch 75/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5260 - accuracy: 0.7322
Epoch 76/100
 98/804 [==>...........................] - ETA: 1s - loss: 0.5223 - accuracy: 0.7283
Epoch 00076: saving model to checkpoints\weights.76.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5258 - accuracy: 0.7324
Epoch 77/100
271/804 [=========>....................] - ETA: 0s - loss: 0.5249 - accuracy: 0.7306
Epoch 00077: saving model to checkpoints\weights.77.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5257 - accuracy: 0.7322
Epoch 78/100
481/804 [================>.............] - ETA: 0s - loss: 0.5281 - accuracy: 0.7315
Epoch 00078: saving model to checkpoints\weights.78.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5259 - accuracy: 0.7319
Epoch 79/100
678/804 [========================>.....] - ETA: 0s - loss: 0.527

In [36]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.5439271330833435, Accuracy: 0.7267638444900513


### Let's try Logistric Regression

In [37]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', random_state=1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.46565597667638486

### Let's try Random Forest

### Ensemble learning

In [38]:
model_ee = EasyEnsembleClassifier(n_estimators=100, random_state=1)
model_ee = model_ee.fit(X_train_scaled, y_train)

In [39]:
predictions_ee = model_ee.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions_ee)

0.699555053950262

# Conclusion
I never obtained any results up to 75%.  I think I must be missing something.  Deep Learning does perform the best but all my variations were very similar in results